### Data Post-procesing: GridRad.

**Read, Filter, Clutter-removal processes for GridRad data based on the modules by the GridRad [website](http://gridrad.org/software.html).**

**For [GridRad - Three-Dimensional Gridded NEXRAD WSR-88D Radar Data](https://rda.ucar.edu/datasets/ds841.0/).**

**Hungjui Yu 20220209**

In [2]:
import sys
import os
import datetime as dt
import pytz
import numpy as np
import netCDF4 as nc
import xarray as xr

In [3]:
def set_input_names(file_date):

    file_path_1_gridrad = '/gpfs/fs1/collections/rda/data/ds841.0'
    # file_path_1_gridrad = '/gpfs/fs1/collections/rda/data/ds841.1'
    file_path_2 = '/{}'.format(file_date.strftime('%Y%m'))

    file_pathnames = dict( dbz = file_path_1_gridrad
                               + file_path_2
                               + '/nexrad_3d_v' + '3_1_' + file_date_time.strftime('%Y%m%dT%H%M%SZ') + '.nc'
                               # + '/nexrad_3d_v' + '4_2_' + file_date_time.strftime('%Y%m%dT%H%M%SZ') + '.nc'
                         )

    
    return file_pathnames

file_date_time = dt.datetime(2011, 4, 27, 12, 0, 0, tzinfo=pytz.utc)
file_name_dict = set_input_names(file_date_time)
print(file_name_dict['dbz'])

/gpfs/fs1/collections/rda/data/ds841.0/201104/nexrad_3d_v3_1_20110427T120000Z.nc


#### Read the GridRad data (original methods):

In [10]:
def read_gridrad(infile):
    
    # Check to see if file exists:
    if not os.path.isfile(infile):
        print('File "' + infile + '" does not exist.  Returning -2.')
        return -2
    
    # Check to see if file has size of zero
    if os.stat(infile).st_size == 0:
        print('File "' + infile + '" contains no valid data.  Returning -1.')
        return -1
    
    # Open GridRad netCDF file
    id = nc.Dataset(infile, "r", format="NETCDF4")
    
    # Read global attributes
    Analysis_time           = str(id.getncattr('Analysis_time'          ))
    Analysis_time_window    = str(id.getncattr('Analysis_time_window'   ))
    File_creation_date      = str(id.getncattr('File_creation_date'     ))
    Grid_scheme             = str(id.getncattr('Grid_scheme'            ))
    Algorithm_version       = str(id.getncattr('Algorithm_version'      ))
    Algorithm_description   = str(id.getncattr('Algorithm_description'  ))
    Authors                 = str(id.getncattr('Authors'                ))
    Project_sponsor         = str(id.getncattr('Project_sponsor'        ))
    Project_name            = str(id.getncattr('Project_name'           ))
    
    # Read list of merged radar sweeps
    if ('sweeps_merged' in id.variables):
        sweeps_list   = (id.variables['sweeps_merged'])[:]
        sweeps_merged = ['']*(id.dimensions['Sweep'].size)
        for i in range(0,id.dimensions['Sweep'].size):
            for j in range(0,id.dimensions['SweepRef'].size):
                sweeps_merged[i] += str(sweeps_list[i,j])
    else:
        sweeps_merged = -1
    

#### Test Run:

In [11]:
read_gridrad(file_name_dict['dbz'])

In [ ]:
%reset